In [1]:
import sys
import os
sys.path.append(os.path.abspath('../src')) # include top level package in python path

In [2]:
from IPython.display import display, HTML
display(HTML("""
<style>
.container { 
    width: 100% !important;
}
</style>
"""))

In [3]:
import torch
from torch import nn, Tensor
from model.fuzzy_layer import (
    FuzzySignedConjunction, 
    FuzzySignedDisjunction, 
    FuzzyUnsignedConjunction, 
    FuzzyUnsignedDisjunction, 
    FuzzyNumKeepup, 
    keepidx, 
    fuzzy_dropup
)
from model.fuzzy_logic import ProductLogic, MinimumLogic, LukasiewiczLogic, DrasticLogic, SchweizerSklarLogic
from model.fuzzy_layer import FuzzyParam
from model.bool_logic import BoolLogic
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from util import shuffle

logic = BoolLogic()
flogic = ProductLogic()

In [4]:
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

trainset = datasets.MNIST(root='./_mnist', train=True, download=True, transform=transforms.ToTensor())
testset = datasets.MNIST(root='./_mnist', train=False, download=True, transform=transforms.ToTensor())

trainloader = DataLoader(trainset, batch_size=128)
testloader = DataLoader(testset, batch_size=128)

In [5]:
class FuzzyMNIST(nn.Module):
    
    def __init__(self, logic):
        super().__init__()
        self.model = nn.Sequential(
            nn.Flatten(),
            nn.Linear(28*28, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.Sigmoid(),
            FuzzySignedConjunction(64, 64, logic=logic),
            FuzzyUnsignedDisjunction(64, 64, logic=logic),
            FuzzySignedConjunction(64, 64, logic=logic),
            FuzzyUnsignedDisjunction(64, 10, logic=logic),
            nn.Softmax(dim=-1),
        )
        
    def forward(self, input):
        return self.model(input)

In [6]:
model = FuzzyMNIST(logic=ProductLogic()).cuda()

In [7]:
def train_loop(model, optim, loader):
    loss_fn = nn.CrossEntropyLoss()
    losses = []
    
    for imgs, labels in loader:
        preds = model(imgs.cuda())
        loss = loss_fn(preds, labels.cuda())
        losses.append(loss.item())

        optim.zero_grad()
        loss.backward()
        optim.step()

    return torch.Tensor(losses)

In [8]:
def test_loop(model, loader):
    correct_count = 0
    total_count = 0
    
    for imgs, labels in loader:
        preds = model(imgs.cuda())
        pred_labels = preds.argmax(dim=-1)
        correct_count += (pred_labels == labels.cuda()).sum()
        total_count += labels.numel()
        
    return correct_count / total_count

In [9]:
def loop(model, optim, trainloader, testloader, epochs=1):
    losses = torch.zeros(0)
    test_accuracies = []
    
    for epoch_no in range(1, epochs+1):
        train_l = tqdm(trainloader, desc="Epoch #%s, Training" % epoch_no, leave=False)
        new_losses = train_loop(model, optim, train_l)
        losses = torch.cat((losses, new_losses), dim=0)
        
        test_l = tqdm(testloader, desc="Epoch #%s, Testing" % epoch_no, leave=False)
        test_accuracies.append(test_loop(model, test_l))
        
    return losses, torch.Tensor(test_accuracies)

In [10]:
optim = torch.optim.Adam(model.parameters(), lr=1e-3)

In [11]:
losses, accuracies = loop(model, optim, trainloader, testloader, epochs=500)

Epoch #1, Training:   0%|          | 0/469 [00:00<?, ?it/s]

Epoch #1, Testing:   0%|          | 0/79 [00:00<?, ?it/s]

Epoch #2, Training:   0%|          | 0/469 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

plt.plot(losses)

In [ ]:
torch.rand(3,4).argmax(dim=-1)

In [ ]:
accuracies

In [ ]:
"-".join(["0","1"])

In [ ]:
[0,1] == [1,0]